In [1]:
import sys
sys.path.append("../Modules/")
sys.path.append("../")
import os

import pandas as pd
import numpy as np

from configobj import ConfigObj
from configobj import flatten_errors
from validate import Validator
    
import logging

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import


In [2]:
%matplotlib notebook

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [3]:
fileTime = '2019-4-25-11-22'
from analysisParameters import loadParameters
analysisParameters = loadParameters(fileTime)

from loadDataE1 import loadSessionDict
sessionDict = loadSessionDict(analysisParameters,
                              loadParsedData = False,
                              loadProcessedData = True)

sessionDict['processed']['blockNumber'] = sessionDict['raw']['blockNumber']
sessionDict['processed']['trialNumber'] = sessionDict['raw']['trialNumber']

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Loading system config file: " + sysCfgPathAndName)? (loadDataE1.py, line 112)

# Get data

In [ ]:
gbTrials = sessionDict['processed'].groupby(('blockNumber','trialNumber'))
trialData = gbTrials.get_group((1,14))
ballHitPaddleOnFr = np.where(trialData['eventFlag']=='ballOnPaddle')[0][0]
row = trialData.iloc[ballHitPaddleOnFr]
np.sqrt(np.sum(np.power(row.ballPos.values - row.paddlePos.values,2)))

In [ ]:
blockNum = 2
trNum = 101

idx = sessionDict['trialInfo'].groupby(('blockNumber','trialNumber')).get_group((blockNum,trNum))['passVertPlaneAtPaddleFr']
fr = sessionDict['trialInfo'].groupby(('blockNumber','trialNumber')).get_group((blockNum,trNum))['passVertPlaneAtPaddleFr']
row = trialData.iloc[fr]
bXYZ = row.ballPos.values
pXYZ = row.paddlePos.values    

#  Ball trajectory direction
ballTrajDir_XYZ = (trialData['ballPos'].iloc[10] - trialData['ballPos'].iloc[1]) / np.linalg.norm(trialData['ballPos'].iloc[10] - trialData['ballPos'].iloc[1])
ballTrajDir_XYZ = np.array(ballTrajDir_XYZ,dtype=np.float)
paddleYDir_xyz = [0,1,0]
paddleXDir_xyz = np.cross(-ballTrajDir_XYZ,paddleYDir_xyz)
# paddleToBallVec_fr_XYZ = trialData['ballPos'].values - trialData['paddlePos'].values

ballRelToPaddle_xyz = np.array(bXYZ-pXYZ).T
xErr = np.float(np.dot(paddleXDir_xyz,ballRelToPaddle_xyz))
yErr = np.float(np.dot(paddleYDir_xyz,ballRelToPaddle_xyz))    

yErr

In [ ]:
blockNum = 2
trNum = 101

idx = sessionDict['trialInfo'].groupby(('blockNumber','trialNumber')).get_group((blockNum,trNum))['passVertPlaneAtPaddleFr']
    
# horizontal vec out of paddle, parallel to ground plane
a = np.array(trialData['paddleToBallDirXZ'].iloc[idx],dtype=np.float)  
b = np.array(trialData.ballPos.iloc[idx] - trialData.paddlePos.iloc[idx],dtype=np.float).T

########################################################
### Calc paddle-to-ball vector

paddleToBallVec_fr_XYZ = trialData['ballPos'].values - trialData['paddlePos'].values
paddleToBallDir_fr_XYZ = np.array([np.divide(XYZ,np.linalg.norm(XYZ)) for XYZ in paddleToBallVec_fr_XYZ],dtype=np.float)

c = paddleToBallDir_fr_XYZ[idx,:] 

X = np.dot(a[0],b)
Y = np.dot(np.array([0,1,0]),b)
Z = np.dot(c,b)


In [ ]:
np.array(trialData['paddleToBallDirXZ'].iloc[idx],dtype=np.float)[0]

In [ ]:
X

In [ ]:
Y

In [ ]:
Z

# Ball error

### Calc ball error for all trials

In [ ]:
gbTrials = sessionDict['processed'].groupby(('blockNumber','trialNumber'))

ballInPaddlePlaneX_fr = []
ballInPaddlePlaneY_fr = []
paddleToBallDist_fr = []

for (blockNum,trNum), trialData in gbTrials:
    
    idx = sessionDict['trialInfo'].groupby(('blockNumber','trialNumber')).get_group((blockNum,trNum))['passVertPlaneAtPaddleFr']
    
#     paddleToBallDir_fr_XYZ = np.array([np.divide(XYZ,np.linalg.norm(XYZ)) for XYZ in paddleToBallVec_fr_XYZ],dtype=np.float)
#     paddleToBallDirXZDf = pd.DataFrame([np.cross([0,1,0],xyz) for xyz in paddleToBallDirDf.values])
    
    a = np.array(trialData['paddleToBallDirXZ'].iloc[idx],dtype=np.float)  # horizontal vec out of paddle, parallel to ground plane
    b = trialData.ballPos.iloc[idx] - trialData.paddlePos.iloc[idx]
    ballInPaddlePlaneX_fr.append(np.dot(a[0],b.values[0]))
    ballInPaddlePlaneY_fr.append(np.dot(np.array([0,1,0]),b.values[0]))
    paddleToBallDist_fr.append(np.sqrt(np.power(ballInPaddlePlaneX_fr[-1],2)+np.power(ballInPaddlePlaneY_fr[-1],2)))
    
    
sessionDict['trialInfo'][('catchingError','X')] = ballInPaddlePlaneX_fr
sessionDict['trialInfo'][('catchingError','Y')] = ballInPaddlePlaneY_fr
sessionDict['trialInfo'][('catchingError','2D')] =  paddleToBallDist_fr

In [ ]:
sessionDict['trialInfo'][('catchingError','X')].iloc[10]

###  Move ball into paddle space

In [ ]:
def paddleToWorldSpace(localBallPos,rowIn):

    mat =[]
    for i in range(16):
        mat.append(rowIn["paddleMat"][str(i)])

    # rearrange and transpose
    transform_4x4 = np.reshape(mat,[4,4]).T
    
    # Take the dot product!
    ballInWorld_XYZ = np.dot(transform_4x4, np.hstack([localBallPos,1]))

    return np.array(ballInWorld_XYZ[0:3])

#######################################################

# Plot ball in paddle coords and calculate error

In [ ]:
# #procData['ballPos'].values[fr-10:fr]
# int(fr)

### Figure settings / layout

In [ ]:

width = 800
height = 800
xRange = [-1,1]
yRange = [0,2]
zRange = [-1,1]

layout = go.Layout(
     scene=go.Scene(aspectmode='manual',
                xaxis=dict(range=xRange),
                yaxis=dict(range=yRange),
                zaxis=dict(range=zRange),
                camera = dict(up=dict(x=0, y=1, z=0)),
                aspectratio=dict(x=1, y=1, z=1),
               ),
    width=width,
    height=height,
    showlegend=False,
   
    margin=go.Margin(t=100),
    hovermode='closest',
                
)

### Plot basis

In [ ]:
blNum = 2
trNum = 21

procData = sessionDict['processed'].groupby(('blockNumber','trialNumber')).get_group((blNum,trNum))
trInfo = sessionDict['trialInfo'].groupby(('blockNumber','trialNumber')).get_group((blNum,trNum))
fr = int(trInfo['passVertPlaneAtPaddleFr'])
row = procData.iloc[fr]

np.sqrt(np.sum(np.power(row.ballPos.values - row.paddlePos.values,2)))

bXYZ = row.ballPos.values
pXYZ = row.paddlePos.values

#  Ball trajectory direction
ballTrajDir_XYZ = (procData['ballPos'].iloc[10] - procData['ballPos'].iloc[1]) / np.linalg.norm(procData['ballPos'].iloc[10] - procData['ballPos'].iloc[1])
paddleYDir_xyz = [0,1,0]
paddleXDir_xyz = np.cross(-ballTrajDir_XYZ,paddleYDir_xyz)
paddleToBallVec_fr_XYZ = procData['ballPos'].values - procData['paddlePos'].values

pX = pXYZ + paddleXDir_xyz
pY = pXYZ + paddleYDir_xyz
pZ = pXYZ + np.cross([0,1,0],paddleXDir_xyz )

# a = np.array(trialData['paddleToBallDirXZ'].iloc[idx],dtype=np.float)  # horizontal vec out of paddle, parallel to ground plane
b = np.array(bXYZ-pXYZ)

xErr = np.dot(paddleXDir_xyz,b) #/ b.values[0]
yErr = np.dot(paddleYDir_xyz,b)
print('Xerror: {:3}'.format(xErr))
print('Yerror: {:3}'.format(yErr))

xsX,ysX,zsX = zip(pX,pXYZ)
xsY,ysY,zsY = zip(pY,pXYZ)
xsZ,ysZ,zsZ = zip(pZ,pXYZ)

paddleCoordsX = go.Scatter3d(x=xsX,
                y=ysX,
                z=zsX,
                mode='lines',
                line = dict(
                  color=('rgb(255,0,0)'),
                  width = 2)
                )

paddleCoordsY = go.Scatter3d(x=xsY,
                y=ysY,
                z=zsY,
                mode='lines',
                line = dict(
                  color=('rgb(0,255,0)'),
                  width = 2)
                )

paddleCoordsZ = go.Scatter3d(x=xsZ,
                y=ysZ,
                z=zsZ,
                mode='lines',
                line = dict(
                  color=('rgb(0,0,255)'),
                  width = 2)
                )

ballPos = go.Scatter3d(x=[bXYZ[0]],
                y=[bXYZ[1]],
                z=[bXYZ[2]],
                mode='markers',
                line = dict(
                  color=('rgb(100,200,100)'),
                  width = 2)
                )

bXYZ_fr = procData.iloc[fr-10:fr+3].ballPos.values
#sessionDict['processed'].iloc[fr-10:fr].ballPos.values
xs,ys,zs = zip(*bXYZ_fr)

ballTraj = go.Scatter3d(x=xs,
                y=ys,
                z=zs,
                mode='markers+lines',
                marker=dict(size=5),
                line = dict(
                  color=('rgb(100,100,100)'),
                  width = 2)
                )

allTraces=[paddleCoordsX,paddleCoordsY,paddleCoordsZ,ballTraj,ballPos]

fig=go.Figure(data=list(allTraces),layout=layout)
iplot(fig)

# Function

In [ ]:

def calcCatchingError(sessionDict):

    ################################################################################
    ### Catching error: passVertPlaneAtPaddleErr X, Y, and 2D

    ballInPaddlePlaneX_fr = []
    ballInPaddlePlaneY_fr = []
    paddleToBallDist_fr = []

    for (blockNum,trNum), tProcData in sessionDict['processed'].groupby(('blockNumber','trialNumber')):
        
        fr = sessionDict['trialInfo'].groupby(('blockNumber','trialNumber')).get_group((blockNum,trNum))['passVertPlaneAtPaddleFr']
        row = tProcData.iloc[fr]
        bXYZ = row.ballPos.values
        pXYZ = row.paddlePos.values

        #  Ball trajectory direction
        ballTrajDir_XYZ = (tProcData['ballPos'].iloc[10] - tProcData['ballPos'].iloc[1]) / np.linalg.norm(tProcData['ballPos'].iloc[10] - tProcData['ballPos'].iloc[1])
        paddleYDir_xyz = [0,1,0]
        paddleXDir_xyz = np.cross(-ballTrajDir_XYZ,paddleYDir_xyz)
        paddleToBallVec_fr_XYZ = tProcData['ballPos'].values - tProcData['paddlePos'].values

        ballRelToPaddle_xyz = np.array(bXYZ-pXYZ).T
        xErr = np.dot(paddleXDir_xyz,ballRelToPaddle_xyz)
        yErr = np.dot(paddleYDir_xyz,ballRelToPaddle_xyz)

        ballInPaddlePlaneX_fr.append(xErr)
        ballInPaddlePlaneY_fr.append(yErr)
        paddleToBallDist_fr.append(np.sqrt(np.power(ballInPaddlePlaneX_fr[-1],2)+np.power(ballInPaddlePlaneY_fr[-1],2)))

    sessionDict['trialInfo'][('catchingError','X')] = ballInPaddlePlaneX_fr
    sessionDict['trialInfo'][('catchingError','Y')] = ballInPaddlePlaneY_fr
    sessionDict['trialInfo'][('catchingError','2D')] =  paddleToBallDist_fr
    
    return sessionDict

sessionDict = calcCatchingError(sessionDict)

In [ ]:
sessionDict['trialInfo'].groupby(('blockNumber','trialNumber')).get_group((2,21))[('catchingError','2D')]


In [ ]:
# Xerror: 0.100759095254
# Yerror: 0.155460000038
